In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [9]:
x_train = np.array([
    [0.312, 0.259, 0.400, 0.216],
    [0.138, 0.311, 0.028, 0.189],
    [0.135, 0.531, 0.431, 0.135],
    [0.314, 0.279, 0.420, 0.256], 
    [0.118, 0.321, 0.528, 0.389],
    [0.185, 0.501, 0.435, 0.175],
    [0.310, 0.255, 0.422, 0.256], 
    [0.238, 0.311, 0.128, 0.289],
    [0.136, 0.532, 0.471, 0.235]
])

y_train = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1])

In [27]:
x_test = ([
    [0.312, 0.259, 0.400, 0.216],
    [0.138, 0.311, 0.028, 0.189],
    [0.135, 0.531, 0.431, 0.135]
])

y_test = np.array([0,0,1])

In [11]:
print(x_train.dtype)
print(y_train.dtype)
print(x_train.shape)
print(y_train.shape)

float64
int32
(9, 4)
(9,)


In [12]:
from torch.utils.data import DataLoader, Dataset, TensorDataset

class custom_data(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        x = torch.tensor(x, dtype = torch.float32)
        y = torch.tensor(y, dtype = torch.long)

        return x, y

In [28]:
train_dataset = custom_data(x_train, y_train)
train_loader = DataLoader(train_dataset)

test_dataset = custom_data(x_test, y_test)
test_loader = DataLoader(test_dataset)

In [14]:
x_in, y_out = next(iter(train_loader))

print(x_in.shape)
print(y_out.shape)

print(x_in.dtype)
print(y_out.dtype)

torch.Size([1, 4])
torch.Size([1])
torch.float32
torch.int64


In [21]:
import torch.nn.functional as F

class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)

        out = self.fc3(x)
        return out

In [22]:
model = Mymodel()

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [30]:
num_epochs = 3500

for epoch in range(num_epochs):
    for x_batch, y_batch in train_loader:
        outputs = model(x_batch)

        optimizer.zero_grad()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss : {loss.item()}, pred : {outputs}')

Epoch 100, Loss : 0.00017248096992261708, pred : tensor([[-4.5714,  4.0935]], grad_fn=<AddmmBackward0>)
Epoch 200, Loss : 0.0001629458274692297, pred : tensor([[-4.5985,  4.1238]], grad_fn=<AddmmBackward0>)
Epoch 300, Loss : 0.00015376816736534238, pred : tensor([[-4.6258,  4.1543]], grad_fn=<AddmmBackward0>)
Epoch 400, Loss : 0.00014506718434859067, pred : tensor([[-4.6531,  4.1847]], grad_fn=<AddmmBackward0>)
Epoch 500, Loss : 0.00013684290752280504, pred : tensor([[-4.6812,  4.2159]], grad_fn=<AddmmBackward0>)
Epoch 600, Loss : 0.00012885693286079913, pred : tensor([[-4.7094,  4.2473]], grad_fn=<AddmmBackward0>)
Epoch 700, Loss : 0.00012134769349358976, pred : tensor([[-4.7380,  4.2790]], grad_fn=<AddmmBackward0>)
Epoch 800, Loss : 0.00011419598013162613, pred : tensor([[-4.7669,  4.3111]], grad_fn=<AddmmBackward0>)
Epoch 900, Loss : 0.00010740180005086586, pred : tensor([[-4.7959,  4.3431]], grad_fn=<AddmmBackward0>)
Epoch 1000, Loss : 0.00010084597306558862, pred : tensor([[-4.825

In [29]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)
    
        _, predicted = torch.max(outputs.data, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f'Accuracy : {100 * correct / total}%')

Accuracy : 66.66666666666667%
